In [10]:
import pdfplumber
import pandas as pd

def extract_table_from_pdf(pdf_path, has_header):
    data = []
    header = None
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            # Extract table settings: Look for lines
            tables = page.extract_tables({
                "vertical_strategy": "lines",
                "horizontal_strategy": "lines"
            })
            
            for table in tables:
                # If header is not set and has_header is True, set header and skip the first row
                if has_header and header is None:
                    header = table[0] + ["page"]
                    start_row_index = 1
                    continue  # Skip the rest of the loop to avoid adding the header as data

                # If header is not set and has_header is False, create a default header
                if header is None:
                    header = [f"column_{i}" for i in range(len(table[0]))] + ["page"]
                    start_row_index = 0


                # Iterate over the table rows, starting from the appropriate index
                for row in table[start_row_index:]:
                    row_with_page = row + [page_num]
                    data.append(row_with_page)
                
    return header, data


council = "East Lothian"
council_mappings = {"Salford": (1, True), "Renfrewshire": (2, True), "East Lothian": (3, False)}
number, has_header = council_mappings[council]
pdf_path = f"sample_pdf{number}.pdf"




In [11]:
column_mappings = {
            "Salford": {"REG": "vrm", "VEHICLE TYPE": "make"},
            "Renfrewshire": {"column_1": "vrm", "column_2": "make"},
            "East Lothian": {
                "column_0": "vrm",
                "column_1": "vrm",
                "column_3": "make",
                "column_4": "make",
                "column_6": "model",
                "column_7": "model",
            },
        }

In [12]:
header, table_data = extract_table_from_pdf(pdf_path, has_header=has_header)

df = pd.DataFrame(table_data, columns=header)

df

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,page
0,,GF16 HJE,,,Vauxhall,,,Octavia S TDi,,1.0
1,DL15 XXT,None,None,Vauxhall,None,None,Astra,None,None,1.0
2,,CP17 YXJ,,,FORD,,,TRANSIT,,1.0
3,MM70 JBV,None,None,Vauxhall,None,None,Corsa,None,None,1.0
4,,GD66 OMC,,,Skoda,,,Octavia Estate,,1.0
...,...,...,...,...,...,...,...,...,...,...
234,SD17 OUF,None,None,Peugeot,None,None,Premier RS Blue HDI S/S,None,None,6.0
235,,SK62 UPV,,,Renault,,,Trafic,,6.0
236,MF68 UYR,None,None,Ford,None,None,Tourneo,None,None,6.0
237,,LM68 LXF,,,Vauxhall,,,Vivaro,,6.0


In [13]:
df.head(50)

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,page
0,,GF16 HJE,,,Vauxhall,,,Octavia S TDi,,1.0
1,DL15 XXT,None,None,Vauxhall,None,None,Astra,None,None,1.0
2,,CP17 YXJ,,,FORD,,,TRANSIT,,1.0
3,MM70 JBV,None,None,Vauxhall,None,None,Corsa,None,None,1.0
4,,GD66 OMC,,,Skoda,,,Octavia Estate,,1.0
5,YE66 UJC,None,None,Renault,None,None,Trafic LL29 Sport Energy\nDCI,None,None,1.0
6,,NK17 DMZ,,,Vauxhall,,,Vivaro Combi,,1.0
7,WF71 EXU,None,None,Hyundai,None,None,IONiQ PREMIUM HEV S-A,None,None,1.0
8,,SC21 WHH,,,Toyota,,,Corolla,,1.0
9,FP15 RDY,None,None,Vauxhall,None,None,Astra Tech Line CDTi S/S,None,None,1.0


In [14]:
def update_headers(column_mappings, df, council):
    column_map = column_mappings[council]
    # Rename the columns using the provided column mapping
    df.rename(columns=column_map, inplace=True)
    
    return df

In [15]:
df = update_headers(column_mappings, df, council)
df.to_csv(f"output_{council}.csv", index=False)

In [16]:
def clean_dataframe(df, column_mappings, council):

    clean_headers = list(set(column_mappings[council].values()))

    print(df)
    print(clean_headers)

    has_null = df[clean_headers].isnull().values.any()
    if has_null:
    
        # Initialize a list to hold the cleaned data
        cleaned_data = []

        # Iterate over the DataFrame rows
        for index, row in df.iterrows():
            # Initialize a dictionary to hold the non-null values for the current row
            non_null_values = {header: None for header in clean_headers}
            
            # Iterate over each header and collect the last non-null value if available
            for header in clean_headers:

                # Get all the values from the columns that were mapped to the current header
                values = row[header].dropna().tolist()
                if values:  # If there are any non-null values
                    # Assign the last non-null value to the corresponding header in the dictionary
                    non_null_values[header] = values[-1]
            
            # Append the dictionary with non-null values to the cleaned data list
            cleaned_data.append(non_null_values)

        # Create a new DataFrame using the cleaned data
        cleaned_df = pd.DataFrame(cleaned_data, columns=clean_headers)

        return cleaned_df
    
    return df

In [17]:
df_clean = clean_dataframe(df, column_mappings, council)

          vrm           vrm      column_2      make      make column_5  \
0                  GF16 HJE                          Vauxhall            
1    DL15 XXT          None          None  Vauxhall      None     None   
2                  CP17 YXJ                              FORD            
3    MM70 JBV          None          None  Vauxhall      None     None   
4                  GD66 OMC                             Skoda            
..        ...           ...           ...       ...       ...      ...   
234  SD17 OUF          None          None   Peugeot      None     None   
235                SK62 UPV                           Renault            
236  MF68 UYR          None          None      Ford      None     None   
237                LM68 LXF                          Vauxhall            
238  SF68 KLO  Ford Pro Cab  Ford Pro Cab         7      None     None   

                       model           model column_8  page  
0                              Octavia S TDi     

In [18]:
df_clean.to_csv(f"output_clean_{council}.csv", index=False)

In [19]:
df_clean

,vrm,model,make
0,GF16 HJE,Octavia S TDi,Vauxhall
1,DL15 XXT,Astra,Vauxhall
2,CP17 YXJ,TRANSIT,FORD
3,MM70 JBV,Corsa,Vauxhall
4,GD66 OMC,Octavia Estate,Skoda
...,...,...,...
234,SD17 OUF,Premier RS Blue HDI S/S,Peugeot
235,SK62 UPV,Trafic,Renault
236,MF68 UYR,Tourneo,Ford
237,LM68 LXF,Vivaro,Vauxhall
